## 最大数
---
**示例1：**
```
输入: [10,2]
输出: 210
```
**示例2：**
```
输入: [3,30,34,5,9]
输出: 9534330
```
**说明：** 输出结果可能非常大，所以你需要返回一个字符串而不是整数。

#### 自定义排序  
#### 想法  
为了构建最大数字，我们希望越高位的数字越大越好。  
#### 算法
首先，我们将每个整数变成字符串。然后进行排序。

如果仅按降序排序，有相同的开头数字的时候会出现问题。比方说，样例 2 按降序排序得到的数字是 9534330395343303 ，然而交换 33 和 3030 的位置可以得到正确答案 95343309534330 。因此，每一对数在排序的比较过程中，我们比较两种连接顺序哪一种更好。我们可以证明这样的做法是正确的：

假设（不是一般性），某一对整数 a 和 b ，我们的比较结果是 a 应该在 b 前面，这意味着 a⌢b>b⌢a ，其中 ⌢ 表示连接。如果排序结果是错的，说明存在一个 c，b 在 c 前面且 c 在 a 的前面。这产生了矛盾，因为a⌢b>b⌢a 和 b⌢c>c⌢b 意味着 a⌢c>c⌢a 。换言之，我们的自定义比较方法保证了传递性，所以这样子排序是对的。



In [6]:
class LargerNumKey(str):
    def __lt__(x, y):
        return x+y > y+x
        
class Solution:
    def largestNumber(self, nums):
        largest_num = ''.join(sorted(map(str, nums), key=LargerNumKey))
        return '0' if largest_num[0] == '0' else largest_num


In [ ]:
###不太对
class Solution:
    def largestNumber(self, nums: List[int]) -> str:
        import functools
        def func(x,y):
            str_x ,str_y= str(x),str(y)
            len_x,len_y =len(str_x),len(str_y)
            for i in range(min(len_x,len_y)):
                if str_x[i]>str_y[i]:
                    return -1
                elif str_x[i]<str_y[i]:
                    return 1
            if len_x<len_y:
                if str_y[i+1]>str_x[i]:
                    return 1
                else:return -1
            elif len_x>len_y:
                if str_y[i]<str_x[i+1]:
                    return -1
                else:return 1
            else: return 0
        if len(set(nums))==1 and nums[0]==0:
            return '0'
        nums=sorted((nums),key=functools.cmp_to_key(func))
        return ''.join(map(str,nums))

## 324.摆动排序II
---
给定一个无序的数组`nums`,将它重新排列成 `nums[0] <nums[1] >nums[2] <nums[3]...`的顺序。
**示例1:**
```
输入: nums = [1, 5, 1, 1, 6, 4]
输出: 一个可能的答案是 [1, 4, 1, 5, 1, 6]
```
**示例2:**
```
输入: nums = [1, 3, 2, 2, 3, 1]
输出: 一个可能的答案是 [2, 3, 1, 3, 1, 2]
```
**说明：**
你可以假设所有输入都会得到有效的结果。
**进阶:**
你能用 O(n) 时间复杂度和 / 或原地 O(1) 额外空间来实现吗？

### 解法1：排序

首先，我们可以很容易想到一种简单的解法：将数组进行排序，然后从中间位置进行等分（如果数组长度为奇数，则将中间的元素分到前面），然后将两个数组进行穿插。

例如：
对于数组[1, 5, 2, 4, 3]，我们将其排序，得到[1, 2, 3, 4, 5]，然后将其分割为[1, 2, 3]和[4, 5]，对两个数组进行穿插，得到[1, 4, 2, 5, 3]。

但是这一解法有一个问题，例如，对于数组[1, 2, 2, 3]，按照这种做法求得的结果仍为[1, 2, 2, 3]。如果题目不要求各元素严格大于或小于相邻元素，即，只要求nums[0] <= nums[1] >= nums[2] <= nums[3]...，那么这一解法是符合要求的，但题目要求元素相互严格大于或小于，那么需要稍微做一点改进。

为了方便阅读，我们在下文中定义较小的子数组为数组A，较大的子数组为数组B。显然，出现上述现象是因为nums中存在重复元素。实际上，由于穿插之后，相邻元素必来自不同子数组，所以A或B内部出现重复元素是不会出现上述现象的。所以，出现上述情况其实是因为数组A和数组B出现了相同元素，我们用r来表示这一元素。而且我们可以很容易发现，如果A和B都存在r，那么r一定是A的最大值，B的最小值，这意味着r一定出现在A的尾部，B的头部。其实，如果这一数字的个数较少，不会出现这一现象，只有当这一数字个数达到原数组元素总数的一半，才会在穿插后的出现在相邻位置。以下举几个例子进行形象地说明：

例如，对于数组[1,1,2,2,3,3],分割为[1,1,2]和[2,3,3]，虽然A和B都出现了2，但穿插后为[1,2,1,3,2,3]，满足要求。
而如果2的个数再多一些，即[1,1,2,2,2,3]，分割为[1,1,2]和[2,2,3]，最终结果为[1,2,1,2,2,3]，来自A的2和来自B的2出现在了相邻位置。

出现这一问题是因为重复数在A和B中的位置决定的，因为r在A尾部，B头部，所以如果r个数太多（大于等于(length(nums) + 1)/2），就可能在穿插后相邻。要解决这一问题，我们需要使A的r和B的r在穿插后尽可能分开。一种可行的办法是将A和B反序：

例如，对于数组[1,1,2,2,2,3]，分割为[1,1,2]和[2,2,3]，分别反序后得到[2, 1, 1]和[3, 2, 2]，此时2在A头部，B尾部，穿插后就不会发生相邻了。

当然，这只能解决r的个数等于(length(nums) + 1)/2的情况，如果r的个数大于(length(nums) + 1)/2，还是会出现相邻。但实际上，这种情况是不存在有效解的，也就是说，这种数组对于本题来说是非法的。

此时我们得到了第一个解法，由于需要使用排序，所以时间复杂度为O(NlogN)，由于需要存储A和B，所以空间复杂度为O(N)。

##### 思路：先把排序后的数组从中间分成2个数组，再把这2个数组分别逆序，最后交叉2个数组成一个数组即可。

In [ ]:
class Solution:
    def wiggleSort(self, nums: List[int]) -> None:
        """
        Do not return anything, modify nums in-place instead.
        """
        import random
        flag = True
        nums.sort()

        lens = len(nums)
        flag = False
        if lens%2==0:len_half = lens//2
        else:
            len_half = lens//2+1
            flag = True
        nums0,nums1 = nums[:len_half], nums[len_half:]
        nums0=nums0[::-1]
        nums1 = nums1[::-1]
        j=0
        if flag==True:len_half-=1
        for i in range(len_half):
            nums[j]=nums0[i]
            j+=1
            nums[j]=nums1[i]
            j+=1
        if flag:
            nums[j]=nums0[-1]
        

### 快速选择算法

### 解法2：快速选择 + 3-way-partition
### [解法3：快速选择 + 3-way-partition + 虚地址](https://leetcode-cn.com/problems/wiggle-sort-ii/solution/yi-bu-yi-bu-jiang-shi-jian-fu-za-du-cong-onlognjia/)

## 寻找峰值
---
峰值元素是指其值大于左右相邻值的元素。
给定一个输入数组 nums，其中 nums[i] ≠ nums[i+1]，找到峰值元素并返回其索引。  
数组可能包含多个峰值，在这种情况下，返回任何一个峰值所在位置即可。  
你可以假设 nums[-1] = nums[n] = -∞。  
**示例1：**
```
输入: nums = [1,2,3,1]
输出: 2
解释: 3 是峰值元素，你的函数应该返回其索引 2。
```
**示例2：**
```
输入: nums = [1,2,1,3,5,6,4]
输出: 1 或 5 
解释: 你的函数可以返回索引 1，其峰值元素为 2；
     或者返回索引 5， 其峰值元素为 6。
```
**说明：**
你的解法应该是 O(logN) 时间复杂度的。

In [ ]:
class Solution:
    def findPeakElement(self, nums: List[int]) -> int:
        if len(nums)==1:
            return 0
        if len(nums)>1 and nums[0]>nums[1]:return 0
        if len(nums)>1 and nums[-1]>nums[-2]:return len(nums)-1
        left = 0
        right = len(nums)-1
        while left <right:
            mid = (left+right)//2
            if nums[mid]>nums[mid-1] and nums[mid]> nums[mid+1]:
                return mid
            elif nums[mid]>nums[mid+1]:
                right = mid
            else:
                left =mid+1

## 287.寻找重复数
---
给定一个包含 `n+1`个整数的数组 nums，其数字都在 `1` 到 `n` 之间（包括 `1` 和 `n`），可知至少存在一个重复的整数。假设只有一个重复的整数，找出这个重复的数。
**示例1：**
```
输入：[1,3,4,2,2]
输出: 2
```
**示例2：**
```
输入：[3,1,3,4,2]
输出：3
```
**说明：**
1. 不能改原数组(假设数组是只读的)。
2. 只能使用额外的 O(1) 的空间。
3. 时间复杂度小于 $ O(n^2) $。
4. 数组中只有一个重复的数字，但它可能不止重复出现一次。

###  快慢指针

In [ ]:
class Solution:
    def findDuplicate(self, nums: List[int]) -> int:
        slow,fast = 0,0
        slow, fast = nums[slow],nums[nums[fast]]
        while slow != fast:slow, fast = nums[slow],nums[nums[fast]]
        slow = 0
        while(slow!= fast):
            slow = nums[slow]
            fast = nums[fast]
        return slow